# ENTREGABLE 1

## 1. Configuración del entorno de trabajo

### Consigna

- Instalar y configurar un sistema de gestión de bases de datos relacional (PostgreSQL o SQL Server).
- Crear una base de datos de trabajo y preparar el entorno para futuras transformaciones (conexión desde Python vía ORM recomendado).

### Detalle de implementación

Se configuró un entorno de base de datos con Docker utilizando **PostgreSQL**. Este servicio ejecuta PostgreSQL y monta los scripts SQL necesarios para crear las tablas al iniciar. Además, se incluye **Adminer** para gestionar la base de datos de forma gráfica.

El entorno está listo para interactuar con la base de datos desde Python usando **SQLAlchemy** como ORM, lo que facilita la implementación de futuras transformaciones y expansiones del proyecto.

---

## 2. Carga inicial de datos

### Consigna

- Crear tablas e importar datos a partir de los archivos CSV proporcionados.
- Ajustar los tipos de datos según corresponda para preservar la integridad semántica y evitar errores de procesamiento.

### Detalle de implementación

La implementación se realizó en **dos fases**:

### Fase 1: Creación de modelos de tablas e importación de datos

Se crearon modelos de tablas basados en las estructuras de los scripts SQL provistos. En esta fase inicial, no se aplicaron restricciones complejas, sino que se enfocó en asegurar la correcta relación entre tablas mediante claves foráneas y los tipos de datos básicos. El objetivo fue importar los datos desde los archivos CSV y asegurar que las claves foráneas (relación entre tablas) estuvieran correctamente configuradas.

**Ejemplo de implementación:**

```python
class Product(BaseModel):

    __tablename__ = TableNames.PRODUCTS

    product_id: Mapped[int] = mapped_column(ProductColumns.PRODUCT_ID, primary_key=True)
    name: Mapped[str] = mapped_column(ProductColumns.NAME, String)
    description: Mapped[str] = mapped_column(ProductColumns.DESCRIPTION, String)
    price: Mapped[Decimal] = mapped_column(ProductColumns.PRICE, Numeric)
    stock: Mapped[int] = mapped_column(ProductColumns.STOCK, Integer)
    category_id: Mapped[int] = mapped_column(
        CategoryColumns.CATEGORY_ID, ForeignKey("Categorias.CategoriaID"), nullable=False
    )
    category: Mapped["Category"] = relationship("Category", backref="products")

    @classmethod
    def from_dict(cls, data: dict):
        return cls(
            product_id=data.get("product_id"),
            name=data.get("name", "").strip(),
            description=data.get("description", "").strip(),
            price=Decimal(data.get("price", 0)),
            stock=data.get("stock", 0),
            category_id=data.get("category_id"),
        )
```
### Fase  2: Ajustes de tipos de datos y restricciones de integridad

Una vez que los datos fueron importados correctamente, se modificaron los modelos para asegurar la integridad semántica de los datos. Esto incluyó:

- Ajuste de tipos de datos: Se cambiaron algunas columnas para asegurar que almacenaran los datos en el tipo adecuado (por ejemplo, fechas o cadenas con longitud fija).

- Aplicación de restricciones de integridad: Se añadieron restricciones como CHECK y UNIQUE para evitar que los datos violaran las reglas de negocio. Esto garantiza que los datos sean válidos y coherentes con las expectativas del sistema.

**Ejemplo de implementación final (con restricciones):**

```python


class Product(BaseModel):

    __tablename__ = TableNames.PRODUCTS

    __table_args__ = (
        CheckConstraint(
            f"{ProductColumns.STOCK} >= 0", name="stock_non_negative"),
        CheckConstraint(
            f"{ProductColumns.PRICE} >= 0", name="price_non_negative"),
    )

    product_id: Mapped[int] = mapped_column(
        ProductColumns.PRODUCT_ID, primary_key=True)
    name: Mapped[str] = mapped_column(ProductColumns.NAME, String(255))
    description: Mapped[str] = mapped_column(
        ProductColumns.DESCRIPTION, String)
    price: Mapped[Decimal] = mapped_column(ProductColumns.PRICE, Numeric)
    stock: Mapped[int] = mapped_column(ProductColumns.STOCK, Integer)
    category_id: Mapped[int] = mapped_column(
        CategoryColumns.CATEGORY_ID,
        ForeignKey("Categorias.CategoriaID"),
        nullable=False,
    )
    category: Mapped["Category"] = relationship("Category", backref="products")

    @classmethod
    def from_dict(cls, data: dict):
        return cls(
            product_id=data.get("product_id"),
            name=data.get("name", "").strip(),
            description=data.get("description", "").strip(),
            price=Decimal(data.get("price", 0)),
            stock=data.get("stock", 0),
            category_id=data.get("category_id"),
        )

```

---

## 3. Tratamiento de campos semi-estructurados


### Consigna

- Identificar columnas que contengan datos en formatos como JSON, listas, o concatenaciones delimitadas.

- Aplicar técnicas de limpieza y transformación para estructurar adecuadamente la información.


### Identificación de columnas con datos semiestructurados

| **Tabla.Columna**               | **Descripción y ejemplo**                                                       | **Decisión de transformación**                                                                                                                                                       |
|----------------------------------|---------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Categorias.Descripción**       | Es una descripción con ejemplos de la misma separados por comas, ej: 'Teléfonos, computadoras, accesorios y dispositivos inteligentes' | No se normaliza, dado que parece una descripción y las preguntas de negocio no requieren ese nivel de detalle. Mantiene su formato como texto.                                        |
| **MetodosPago.Descripcion**     | Caso similar al de `Categorias.Descripción`, ej: 'Pago con tarjeta de crédito VISA, MasterCard o American Express'                  | No se aplicó transformación, dado que no se requiere conocer marcas de tarjetas, solo la modalidad de pago (ej., 'Tarjeta de Crédito', 'Transferencia Bancaria').                   |

---

## 4.Análisis exploratorio y evaluación de calidad de datos

### Consigna


- Explorar la estructura y el contenido de los datos utilizando consultas SQL.

- Implementar análisis exploratorio complementario en Python mediante un ORM (como SQLAlchemy o psycopg2).

- Detectar valores nulos, duplicados, atípicos y otras inconsistencias.

- Identificar claves primarias y foráneas implícitas, atributos principales y variables relevantes para el negocio.

- Proponer y documentar acciones de preprocesamiento y corrección de calidad de datos.

### Desarrollo de consignas 

#### 1. Explorar la estructura y el contenido de los datos utilizando consultas SQL
En esta fase del análisis, el objetivo principal es obtener una comprensión detallada sobre la estructura y el contenido de las tablas dentro de la base de datos, así como la naturaleza de los datos. Este paso es fundamental porque nos permite identificar las relaciones entre las tablas, la calidad de los datos, y cualquier posible inconsistencia que pueda requerir correcciones antes de realizar análisis más avanzados.


- Consultas de exploración

Consulta para identificar tablas

In [ ]:
from app.utils import postgres_utils

query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""
tables = postgres_utils.run_query(query)

tables

* Consulta de la estructura de las tablas , para conocer columnas y tipos de datos de las mismas.

In [ ]:

from app.utils import postgres_utils

query_tables_structure = """
        SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = 'public'
        ORDER BY table_name, ordinal_position;
    """

postgres_utils.run_query(query_tables_structure)

Consulta para obtener primeros registros de cada tabla, para explorar los datos y formatos:

In [ ]:
from app.utils import postgres_utils
import logging

logging.getLogger("sqlalchemy.engine").setLevel(logging.WARNING)

def get_first_records(table_name):
    query = f"SELECT * FROM \"{table_name}\" LIMIT 10;"
    data = postgres_utils.run_query(query)
    return data

tables_list = tables['table_name'].unique()
for table in tables_list:
    data= postgres_utils.run_query(f"SELECT * FROM \"{table}\" LIMIT 5;")
    display(data)


## 2.Implementar análisis exploratorio complementario en Python mediante un ORM (como SQLAlchemy o psycopg2).

En [Archivo Entrega 1](1st_delivery.ipynb)
